In [3]:
!export HF_ENDPOINT=https://hf-mirror.com
!export CUDA_VISIBLE_DEVICES=0

In [4]:
# package import
from torch import Tensor
from transformer_lens import utils
from functools import partial
from jaxtyping import Int, Float
import torch
# device setup
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

Device: cuda


In [5]:
device="cpu"

In [6]:
from dotenv import load_dotenv
load_dotenv("/home/ckqsudo/code2024/CKQ_ACL2024/Control_Infer/SAE-simple/.env")

True

In [7]:
from transformer_lens import HookedTransformer
import numpy as np
import torch
import plotly_express as px

from transformer_lens import HookedTransformer

# Model Loading

from sae_lens.analysis.neuronpedia_integration import get_neuronpedia_quick_list

# Virtual Weight / Feature Statistics Functions
from sae_lens.analysis.feature_statistics import (
    get_all_stats_dfs,
    get_W_U_W_dec_stats_df,
)

# Enrichment Analysis Functions
from sae_lens.analysis.tsea import (
    get_enrichment_df,
    manhattan_plot_enrichment_scores,
    plot_top_k_feature_projections_by_token_and_category,
)
from sae_lens.analysis.tsea import (
    get_baby_name_sets,
    get_letter_gene_sets,
    generate_pos_sets,
    get_test_gene_sets,
    get_gene_set_from_regex,
)
import os
os.environ['HF_ENDPOINT']="https://hf-mirror.com"
# model = HookedTransformer.from_pretrained(
#     "tiny-stories-1L-21M"
# )  # This will wrap huggingface models and has lots of nice utilities.


model = HookedTransformer.from_pretrained("gpt2-small",device=device)

Loaded pretrained model gpt2-small into HookedTransformer


In [8]:
from sae_lens import SAE
layer=5

gpt2_small_sparse_autoencoders = {}
gpt2_small_sae_sparsities = {}


sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="gpt2-small-res-jb",  # see other options in sae_lens/pretrained_saes.yaml
    sae_id=f"blocks.{layer}.hook_resid_pre",  # won't always be a hook point
    device=device
)
gpt2_small_sparse_autoencoders[f"blocks.{layer}.hook_resid_pre"] = sae
gpt2_small_sae_sparsities[f"blocks.{layer}.hook_resid_pre"] = sparsity

/home/ckqsudo/miniconda3/envs/SAE/lib/python3.11/site-packages/sae_lens/sae.py:145: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(


In [9]:
from datasets import load_dataset
dataset = load_dataset("/home/ckqsudo/code2024/0dataset/emotional_classify/multiclass-sentiment-analysis")

In [10]:
dataset["train"]= dataset['train'].shuffle(seed=42)  # seed 用于固定随机性



In [11]:
train_data_text=dataset["train"]# 假设数据集是训练集（train），筛选 labels = 1 的数据
neg_train_set = dataset['train'].filter(lambda example: example['label'] == 0).select(range(1000))

pos_train_set=dataset['train'].filter(lambda example: example['label'] == 2).select(range(1000))

In [12]:
neu_train_set=dataset['train'].filter(lambda example: example['label'] == 1).select(range(1000))

In [13]:
len(neg_train_set),len(pos_train_set)

(1000, 1000)

In [14]:
neg_train_set["text"][1],pos_train_set["text"][1]

('needs someone to explain lambda calculus to him! :(', 'Amazing!!!!')

In [15]:
hook_point = sae.cfg.hook_name

In [16]:
sv_logits, cache = model.run_with_cache(pos_train_set["text"][:1000], prepend_bos=True,device=device)

In [17]:
sv_logits.shape

torch.Size([1000, 177, 50257])

In [18]:
batch_hidden_states=cache[hook_point]

In [19]:
batch_hidden_states.device

device(type='cpu')

In [20]:
batch_hidden_states.shape

torch.Size([1000, 177, 768])

In [21]:
import torch

# 定义一个三维 Tensor
tensor = torch.tensor([
    [
    [0, 1, 0, 2],
    [3, 0, 0, 4],
    [0, 0, 0, 0]],

                       [[5, 0, 6, 0],
                        [0, 7, 0, 8],
                        [9, 0, 0, 0]]])

# 统计在 A 和 B 两个维度上不为 0 的元素，输出 C 维度的向量
nonzero_counts_C = (tensor != 0).sum(dim=(0, 1))
# # 1. 统计非零元素的数量
# latents_count = (batch_latents != 0).sum(dim=(0, 1))

# 打印结果
print(nonzero_counts_C,tensor.shape)

tensor([3, 2, 1, 3]) torch.Size([2, 3, 4])


: 

In [20]:
from matplotlib import axis
import torch
batch_latents=[]
for data_idx in range(0,batch_hidden_states.shape[0]):
    hidden_state=batch_hidden_states[0]
    # hidden_state.to("cuda")
    latents = sae.encode(hidden_state)
    batch_latents.append(latents)
batch_latents=torch.stack(batch_latents,axis=0)


In [21]:
# import torch

# # 定义矩阵
# matrix = torch.zeros(4,2)
# matrix[2,1]=4
# matrix[3,1]=9
# matrix[1,0]=-1
# # 统计每一列中不为 0 的元素的个数
# nonzero_counts = (matrix != 0).sum(dim=0)

# # 打印结果
# print(nonzero_counts)

In [22]:
batch_latents.shape

torch.Size([1000, 177, 24576])

In [23]:
batch_latents.shape

torch.Size([1000, 177, 24576])

In [24]:

# 1. 统计非零元素的数量
act_cnt = (batch_latents != 0).sum(dim=(0, 1))

# 2. 计算非零元素的总和
nz_sum = torch.where(batch_latents != 0, batch_latents, torch.tensor(0.0)).sum(dim=(0, 1))

# 3. 计算非零元素的均值
nz_mean = torch.where(act_cnt != 0, nz_sum / act_cnt, torch.tensor(0.0))

In [25]:
torch.max(act_cnt )

tensor(151000)

In [26]:
nz_mean[12555]

tensor(0., grad_fn=<SelectBackward0>)

In [27]:
nz_act_val, nz_val_indices= torch.topk(nz_mean, 100)

In [28]:
nz_act_val,nz_val_indices

(tensor([519.3588, 400.9080, 388.5621, 319.9220, 282.2983, 231.8186, 204.4063,
          52.3016,  47.9561,  41.2254,  40.8973,  36.4871,  33.9533,  33.3195,
          32.2983,  30.6149,  27.8049,  26.2022,  22.9452,  21.7568,  19.3381,
          19.2138,  19.1500,  18.1030,  17.9155,  16.4459,  13.8892,  13.8246,
          13.7617,  12.6209,  12.4772,  12.2763,  12.0794,  11.1112,  11.0233,
          10.7947,  10.5119,  10.4058,  10.3053,  10.2840,  10.0442,   9.9688,
           9.8907,   9.8810,   9.8516,   9.7237,   9.7208,   9.6545,   9.5016,
           9.1099,   8.5513,   8.2379,   8.1908,   8.1376,   8.1075,   7.8428,
           7.7397,   7.6669,   7.6608,   7.6127,   6.7918,   6.7808,   6.7535,
           6.6709,   6.6701,   6.6061,   6.5835,   6.5640,   6.5369,   6.5007,
           6.4865,   6.4255,   6.2286,   6.1316,   6.0690,   5.8662,   5.8205,
           5.8132,   5.7220,   5.7059,   5.5783,   5.5024,   5.4399,   5.3980,
           5.3934,   5.3803,   5.3785,   5.3189,   5

In [29]:
nz_val_indices

tensor([ 1334, 14618, 16949,  4242,  1885, 12411,  1256, 14548,   859, 21614,
        13978, 13928, 23393,  3188,   303, 13552, 11355,  4536,  4348, 10466,
        15338, 21865, 11217,  5697, 17590,  6417,  3465, 22297,  6108, 14375,
        24559,  4776, 21518,  9436, 20599,  6627,  5389,  3932, 23333, 11154,
        15622, 10304,  1268, 11066, 12915,   317, 18372, 17268,  6778, 23768,
        21189, 21573, 15694, 15583,  1660, 14916, 21211, 12700, 23183,  4166,
        18106,  2358,  6648, 13018, 15029,  7135, 11502,  7241, 12773,  2678,
         1996, 21819,  4530, 12410,  1371, 19687, 23965, 20688,  6944, 23255,
          658, 23866,  9329, 10981, 22668, 16623, 10140,  3829, 14982, 14147,
        11945, 15519, 16563, 24223, 14585,  3705,   682, 15627,  7254,  8173])

In [30]:
nz_cnt, cnt_indices = torch.topk(act_cnt, 200)

# nonzero_indices = torch.nonzero(latents_count > 0).squeeze()

In [31]:
overlap_indices=nz_val_indices[torch.isin(nz_val_indices,cnt_indices)]

In [32]:
len(overlap_indices)

10

In [33]:
import torch

# 创建两个 Tensor
tensor1 = torch.tensor([1, 2, 3, 4, 5])
tensor2 = torch.tensor([4, 5, 6, 7, 8,9])

# 检查 tensor1 中的元素是否在 tensor2 中
mask = torch.isin(tensor1, tensor2)

# 提取重复元素
intersection = tensor1[mask]

# 打印结果
print("重复元素:", intersection,mask.shape)

重复元素: tensor([4, 5]) torch.Size([5])


In [34]:
def check_acts_by_index(prompt,acts_idx,hook_point):
    sv_logits, cache = model.run_with_cache(prompt, prepend_bos=True)
    # 转换为对应的令牌表示
    tokens = model.to_tokens(prompt)
    print(tokens)

    # get the feature activations from our SAE 获取特定位置的中间状态,并对中间状态进行编码，得到特征激活
    sv_feature_acts = sae.encode(cache[hook_point])
    print(sv_feature_acts.shape)
    return sv_feature_acts[:,:,acts_idx-1:acts_idx+1]


In [35]:
sae.W_dec.shape

torch.Size([24576, 768])

In [36]:
method="val_mul"

In [37]:
def steering_vectors(method):
    if method=="mean":
        steering_vectors=Tensor.mean(sae.W_dec[overlap_indices],axis=0)
    elif method=="val_mul":
        steering_vectors=torch.zeros(768)
        for i,important_idx in enumerate(overlap_indices):
            assert nz_mean[important_idx]>2
            steering_vectors+=nz_mean[important_idx]*sae.W_dec[important_idx]
    return steering_vectors
            
# d_hidden=sae.W_dec.shape[1]
# d_latent=sae.W_dec.shape[0]

In [38]:
mean_steering_vec=steering_vectors("mean")
mul_steering_vec=steering_vectors("val_mul")

In [39]:
"欧几里得距离",torch.norm(mean_steering_vec - mul_steering_vec, p=2)

('欧几里得距离', tensor(81.0290, grad_fn=<LinalgVectorNormBackward0>))

In [40]:
import torch.nn.functional as F
"余弦",F.cosine_similarity(mean_steering_vec.unsqueeze(0), mul_steering_vec.unsqueeze(0))

('余弦', tensor([0.9130], grad_fn=<SumBackward1>))

In [41]:
steering_vectors=mul_steering_vec
print("注意你最后选择的干预向量",method)


注意你最后选择的干预向量 val_mul


In [42]:
# 

from ast import Raise


def steering_hook(resid_pre, hook):
    if resid_pre.shape[1] == 1:
        return

    # position = steer_prompt_id.shape[1]
    # raise ValueError(f"steering{resid_pre.shape}")
    if steering_on:
        # using our steering vector and applying the coefficient
        # position=steer_prompt_id.shape[-1]
        resid_pre[:, :, :] += coeff * steering_vectors


def hooked_generate(prompt_batch, fwd_hooks=[], seed=None, **kwargs):
    if seed is not None:
        torch.manual_seed(seed)

    with model.hooks(fwd_hooks=fwd_hooks):
        tokenized = model.to_tokens(prompt_batch)
        result = model.generate(
            stop_at_eos=True,  # avoids a bug on MPS
            input=tokenized,
            max_new_tokens=50,
            do_sample=True,
            **kwargs,
        )
    return result
def run_generate(example_prompt):
    model.reset_hooks()
    editing_hooks = [(f"blocks.{layer}.hook_resid_post", steering_hook)]
    res = hooked_generate(
        [example_prompt] * 3, editing_hooks, seed=None, **sampling_kwargs
    )# batch=3, e

    # Print results, removing the ugly beginning of sequence token
    res_str = model.to_string(res[:, 1:])
    print(("\n\n" + "-" * 80 + "\n\n").join(res_str))

In [43]:
sampling_kwargs = dict(temperature=1.0, top_p=0.5, freq_penalty=1.0)

In [44]:
example_prompt=neu_train_set["text"][1]

In [45]:
example_prompt

'Is listening to the new jonas brothers song'

In [46]:
steering_on=False
example_prompt="I feel"
coeff=0
run_generate(example_prompt=example_prompt)

  0%|          | 0/50 [00:00<?, ?it/s]

I feel like this is a fun little story. I was thinking about this from the start, but then I realized that this one had to be on my to-do list.

It's not as if there's a huge amount of extra work involved

--------------------------------------------------------------------------------

I feel like this article is a reflection of the lack of support I've received for this project. It has become my default site, and I'm very disappointed in it.

What I've been able to accomplish so far is just to build something that

--------------------------------------------------------------------------------

I feel like this is a really good way to do it. I just don't have any of the time or energy for it.

The last thing I want is to be distracted from my day-to-day life and focus on other things,


In [66]:
steering_on=True
# example_prompt="hey! I feel"
coeff=10
run_generate(example_prompt=example_prompt)

  0%|          | 0/50 [00:00<?, ?it/s]

I feelShare is in a state of flux and should be up for renewal, so if you're interested in seeing the band return to its roots, you can get your hands on their new album from December 11th.

See more music news and reviews

--------------------------------------------------------------------------------

I feelThe least bit guilty for what I did, but I've got a message for you guys. The first thing that happened to me was when someone posted on the comments section of this thread about my wife and how much she loves her man. My wife

--------------------------------------------------------------------------------

I feelPosted

"Worse than the fire and the death of my children, where can I find an excuse to call them 'themselves' again?"


When asked about his time in prison, he replied "You know, we're all
